In [1]:
import os
from os.path import join
import time
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import torch
import torch.multiprocessing as mp
from torch.distributed import init_process_group, destroy_process_group
from torch.utils.data.distributed import DistributedSampler
from torch.utils.data import DataLoader
from astropy.io import fits
import pyxis.torch as pxt

from networks import ForkCNN
from train import *
from dataset import FiberDataset
import config

data_dir = '/xdisk/timeifler/wxs0703/kl_nn/train_data_massive_sf/train_database'
samp_dir = '/xdisk/timeifler/wxs0703/kl_nn/samples/samples_massive_static_fibers.csv'
fig_dir = '/xdisk/timeifler/wxs0703/kl_nn/figures/'
model_dir = '/xdisk/timeifler/wxs0703/kl_nn/model/'

In [2]:
os.environ["MASTER_ADDR"] = "localhost"
os.environ["MASTER_PORT"] = "12355"
torch.cuda.set_device(0)
init_process_group(backend='nccl', rank=0, world_size=1)

In [3]:
ds = pxt.TorchDataset(data_dir)

In [4]:
save_every = 1
nepochs = 20
batch_size = 100

In [5]:
dl = DataLoader(
        ds,
        batch_size=batch_size,
        pin_memory=True,
        shuffle=False,
        sampler=DistributedSampler(ds),
        num_workers=1,
    )

In [6]:
model = ForkCNN(batch_size)
model.to(0)

ForkCNN(
  (cnn_img): Sequential(
    (0): Conv2d(1, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU(inplace=True)
    (3): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    (4): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (5): ReLU(inplace=True)
    (6): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (7): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    (8): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (9): ReLU(inplace=True)
    (10): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    (11): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (12): ReLU(inplace=True)
    (13): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation

In [8]:
start = time.time()
for i, batch in enumerate(dl):
    if i == 0:
        img = batch['img'].float().to(0)
        spec = batch['spec'].float().to(0)
        fid = batch['fid_pars'].float().view(-1,8).to(0)
        print(img.shape, spec.shape, fid.shape)
        out = model.forward(img, spec)
    print(f'Batch {i} finished')
    if i==200:
        break
t = time.time()-start
print(t)

torch.Size([100, 1, 48, 48]) torch.Size([100, 1, 3, 64]) torch.Size([100, 8])


/home/u2/wxs0703/.local/lib/python3.9/site-packages/torch/nn/modules/conv.py:456: UserWarning: Plan failed with a cudnnException: CUDNN_BACKEND_EXECUTION_PLAN_DESCRIPTOR: cudnnFinalize Descriptor Failed cudnn_status: CUDNN_STATUS_NOT_SUPPORTED (Triggered internally at ../aten/src/ATen/native/cudnn/Conv_v8.cpp:919.)
  return F.conv2d(input, weight, bias, self.stride,


Batch 0 finished
Batch 1 finished
Batch 2 finished
Batch 3 finished
Batch 4 finished
Batch 5 finished
Batch 6 finished
Batch 7 finished
Batch 8 finished
Batch 9 finished
Batch 10 finished
Batch 11 finished
Batch 12 finished


KeyboardInterrupt: 

In [9]:
destroy_process_group()

In [2]:
world_size = torch.cuda.device_count()
save_every = 1
nepochs = config.train['epoch_number']
batch_size = config.train['batch_size']
nfeatures = config.train['feature_number']

In [3]:
mp.spawn(train_nn, args=(world_size, save_every, nepochs, batch_size, nfeatures), nprocs=world_size)

/home/u2/wxs0703/.local/lib/python3.9/site-packages/torch/nn/modules/conv.py:456: UserWarning: Plan failed with a cudnnException: CUDNN_BACKEND_EXECUTION_PLAN_DESCRIPTOR: cudnnFinalize Descriptor Failed cudnn_status: CUDNN_STATUS_NOT_SUPPORTED (Triggered internally at ../aten/src/ATen/native/cudnn/Conv_v8.cpp:919.)
  return F.conv2d(input, weight, bias, self.stride,
/home/u2/wxs0703/.local/lib/python3.9/site-packages/torch/nn/modules/conv.py:456: UserWarning: Plan failed with a cudnnException: CUDNN_BACKEND_EXECUTION_PLAN_DESCRIPTOR: cudnnFinalize Descriptor Failed cudnn_status: CUDNN_STATUS_NOT_SUPPORTED (Triggered internally at ../aten/src/ATen/native/cudnn/Conv_v8.cpp:919.)
  return F.conv2d(input, weight, bias, self.stride,
/home/u2/wxs0703/.local/lib/python3.9/site-packages/torch/autograd/graph.py:744: UserWarning: Plan failed with a cudnnException: CUDNN_BACKEND_EXECUTION_PLAN_DESCRIPTOR: cudnnFinalize Descriptor Failed cudnn_status: CUDNN_STATUS_NOT_SUPPORTED (Triggered internal

KeyboardInterrupt: 